# Osnovi računarske inteligencije

## Vežba 1 - Upoznavanje sa okruženjem za pretrage

Tokom narednih nekoliko termina vežbi bavićemo se problemima pretrage.

Kako bismo vizualizovali različite probleme i algoritme pretrage, koristićemo "robot" okruženje.


### Datoteke i šta se nalazi u njima
- `board.py` - klasa za čuvanje informacija o trenutnoj tabli i njeno menjanje
- `game.py` - korisnički interfejs i logika igre "robot"
- `main.py` - skripta za pokretanje okruženja
- `search.py` - implementacija različitih algoritama pretrage
- `state.py` - klasa za reprezentaciju stanja problema pretrage

Najvažnije su nam datoteke `search.py` i `state.py`

### Problem pretrage

Da bismo razumeli kod, neophodno je prvo da razumemo šta je problem pretrage.

Pretraga se vrši nad nekim prostorom stanja.

Prostor stanja je graf gde su čvorovi konkretna jednistvena stanja problema, a grane akcije kojima se iz jednog stanja prelazi u drugo.

<img src="img/prostor_stanja.png" width="400"/>

Cilja pretrage je pronaći (optimalnu) putanju od početnog do ciljanog stanja.

#### Definicija stanja

Za potrebe pretraga, jedno stanje treba da definiše:
- Da li je trenutno stanje ciljano stanje
- Koja stanja mogu da sleduju iz trenutnog stanja
- Da li je stanje jednako sa nekim drugim stanjem - funkcija poređenja
- Roditeljsko stanje - za rekonstrukciju putanje

Kod vođenih pretraga, stanje dodatno definiše
- Svoju trenutnu cenu - `g(n)`
- Heurističku funkciju svoje udaljenosti od ciljanog stanja - `h(n)`

### `search.py`
Sadrži implementacije različitih algoritama pretrage.

Svi algoritmi pretrage imaju zajednički generički deo, a razlikuju se samo po načinu odabira stanja - više o ovome na sledećem terminu vežbi.

Algoritmi su implementirani kao klase koje nasleđuju `Search` klasu i redefinišu metodu `select_state`.

---

### `state.py`

Sadrži klase `State` i `RobotState` koje predstavljaju jedno stanje iz prostora stanja koje pretražujemo. Ovde se nalazi definicija našeg problem pretrage - šta je cilj pretrage i kako stižemo do njega. Npr. cilj može biti stići do cvene kutije, a može biti i prvo pokupiti plavu kutiju, pa zatim stići do crvene kutije.

`State` je apstraktna klasa koja inicijalizuje neke vrednosti stanja i sadrži apstraktne funkcije za koje se očekuje da budu implementirane u klasi naslednici. Ovu klasu nije potrebno menjati za rešavanje zadataka, ali je dobro razumeti koje sve vrednosti se u njoj inicijalizuju.
- `board` je vrednost koja se uvek prosleđuje kroz konstruktor, i tipa je `Board` iz `board.py`
- `parent` je vrednost roditeljskog stanja (stanja iz kog smo došli). Ako je None, to znači da je trenutno stanje početno.
- `position` - pozicija robota, koja se pronalazi iz table ukoliko nije prosleđena (što je slučaj samo za početno stanje).
- `goal_position`- pozicija crvenog kvadrata na tabli (označava cilj), dobavlja se iz table ukoliko je stanje početno, u suprotnom se prosleđuje od roditeljskog stanja ka detetu. 

`State` klasa takođe sadrži implementaciju funkcije `get_next_states`. Ova funckija vraća listu susednih stanja, a oslanja se na funkciju `get_legal_positions`. Dakle, `get_next_states` će od niza mogućih pozicija da napravi niz objekata tipa State. Ovime osiguravamo da se konstruktor od `State` ispravno poziva.

Pored toga, `State` klasa definiše funkcije `get_agent_code`, `get_agent_goal_code`, `get_checkpoint_code` i `get_teleport_code`koje vraćaju kodove za različite elemente igre (robot, cilj, itd.) - koriste se u konstruktoru roditeljske klase za pronalazak  pozicija elemenata sa table.

`RobotState` nasleđuje `State` i u njoj rešavamo većinu zadataka. Služi kako bismo definisali konkretan problem pretrage koji rešavamo. Sadrži funkcije u kojima implementiramo delove rešenja:
- `__init__` - konstruktor. Ovde vršimo inicijalizaciju svega dodatnog što nam treba za rešavanje problema. Nije dobra ideja proširivati konstruktor sa dodatnim parametrima. Ukoliko želimo da pokupimo neku informaciju od roditelja, to možemo da uradimo kroz `self.parent` atribut. Npr. implementacija brojanja poteza bi izgledala ovako:

In [ ]:
if self.parent is None:
    self.steps = 0
else:
    self.steps = self.parent.steps + 1


- `get_legal_positions` - funckija prelaska stanja - treba da vrati niz pozicija (x, y) koja mogu biti naredni položaji robota na tabli nakon odrađene akcije
- `is_final_state` - vraća *boolean* vrednost da li je trenutno stanje ciljano stanje
- `unique_hash` - treba da vrati jednistvenu vrednost (*string*) koja će da služi da razlikuje ovo stanje od svih ostalih stanja
- `get_current_cost` - treba da vrati cenu dosadašnje putanja pređene da bi robot stigao do ovog stanja
- `get_cost_estimate` - treba da vrati procenu preostale putanje do ciljanog stanja


#### Zadaci

**Napomena**: Instalirati potrebne biblioteke: 
``` 
pip install --upgrade Pillow
```

TK instalacija [link](https://tkdocs.com/tutorial/install.html#install-x11-python) 

Otvoriti projekat (u *src/robot*), i pokrenuti program kroz razvojno okruženje ili kroz komandnu liniju (```python main.py```). Program se sastoji iz table, robota i robotovog cilja (crveno polje). Potrebno je da robot/agent algoritmom pretrage pronađe put do cilja, zaobilazeći prepreke (zidove). Klikom na određenu ćeliju table može se menjati sadržaj ćelije - može biti **robot, cilj (crveno), zid (sivo), kutija(plavo), portal(žuto)**. Desni klik menja polja u obrnutom redosledu. Trenutno je implementirana jedna strategija pretrage (prvi u širinu - *BFS*). Moguće je učitati i sačuvati postavke table pomoću `File/Open...` i `File/Save...`.

Robota možemo da pomeramo na strelice na tastaturi.

Klikom na dugme *search* pokreće se pretraga, a kao rezultat se brojevima obeležavaju redom polja kojima robot treba da se kreće. U konzoli se ispisuje uspešnost pretrage.

---

##### Proširenja stanja

**TODO 1**: Izmeniti definiciju problema pretrage tako da agent prvo pokupi plavu kutiju, a zatim stigne do cilja.

Koraci:
- u *init* funkciji definisati atribut *has_box* i postaviti ga na *False*. Ukoliko se na trenutnoj poziciji robota nalazi kutija, postaviti *has_box* na *True*
- u *is_final_state* funkciji dodati proveru da li je robot pokupio kutiju
- proširiti *unique_hash* funckiju da uključuje i informaciju o tome da li robot ima kutiju

---

##### DODATNO

Izmeniti definiciju problema pretrage tako da agent prvo pokupi sve plave kutiju, a zatim stigne do cilja.